# Clustering Neighborhoods

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
import requests
import random

In [3]:
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 

In [4]:
import folium

# Initializing values

In [5]:
client_id = 'ITKVISZUN50K5OIU5THJ0A330ABDVLFMOSX4NUB41ESFVYSA'
client_secret = 'MACPFVUEFU3BEHD4YZREQUBDEBFX0JFOPQ0YLIEBRHWQ0ELC'
ver = '20180604'
lim = 30

In [6]:
addr = '102 North End Ave, New York, NY'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(addr)
lat = location.latitude
long = location.longitude
print(lat,long)

40.7151482 -74.0156573


In [7]:
search_q = 'Italian'
radius = 500

In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(client_id, client_secret, lat, long, ver, search_q, radius, lim)
url

'https://api.foursquare.com/v2/venues/search?client_id=ITKVISZUN50K5OIU5THJ0A330ABDVLFMOSX4NUB41ESFVYSA&client_secret=MACPFVUEFU3BEHD4YZREQUBDEBFX0JFOPQ0YLIEBRHWQ0ELC&ll=40.7151482,-74.0156573&v=20180604&query=Italian&radius=500&limit=30'

In [9]:
data = requests.get(url).json()
data

{'meta': {'code': 200, 'requestId': '5f0d58305bd87379b3f26584'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 77,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15947104

In [10]:
# assigning venue data
venue = data['response']['venues']

# creating df
df = json_normalize(venue)
df.head()

<ipython-input-10-273628fa3cdf>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(venue)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1594710487,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",77,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1594710487,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",499,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


# Cleaning Data

In [11]:
fil_cols = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filter = df.loc[:, fil_cols]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filter['categories'] = df_filter.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filter.columns = [column.split('.')[-1] for column in df_filter.columns]
df_filter

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",77,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",499,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",4f3232e219836c91c7bfde94


# Visualizing location

In [80]:
venue_m = folium.Map(location=[lat,long], zoom_start=15)

folium.CircleMarker(
    [lat,long],
    radius=10,
    popup='Conrad Hotel',
    fill=True,
    fill_color='red',
    fill_opacity=0.4
).add_to(venue_m)
for lat, lng, label in zip(df_filter.lat, df_filter.lng, df_filter.categories):
    folium.CircleMarker(
    [lat,lng],
    radius=10,
    fill=True,
    popup=label,
    fill_color='purple',
    fill_opacity=0.5).add_to(venue_m)
venue_m

# Exploring Ratings of Venues

In [19]:
# Harry's Italian Pizza Bar
ven_id = '4fa862b3e4b0ebff2f749f06'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(ven_id,client_id,client_secret,ver)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=ITKVISZUN50K5OIU5THJ0A330ABDVLFMOSX4NUB41ESFVYSA&client_secret=MACPFVUEFU3BEHD4YZREQUBDEBFX0JFOPQ0YLIEBRHWQ0ELC&v=20180604'

In [26]:
data2 = requests.get(url).json()
print(data2['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


In [24]:
print('Rating:', data2['response']['venue']['rating'])

Rating: 6.9


In [27]:
# Conca Cucina hotel
ven_id = '4f3232e219836c91c7bfde94'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(ven_id,client_id,client_secret,ver)
url

'https://api.foursquare.com/v2/venues/4f3232e219836c91c7bfde94?client_id=ITKVISZUN50K5OIU5THJ0A330ABDVLFMOSX4NUB41ESFVYSA&client_secret=MACPFVUEFU3BEHD4YZREQUBDEBFX0JFOPQ0YLIEBRHWQ0ELC&v=20180604'

In [30]:
data3 = requests.get(url).json()
print(data3['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


In [32]:
try:
    print('Rating',data3['response']['venue']['rating'])
except:
    print('No rating available')

No rating available


In [34]:
# For Ecco Hotel
ven_id = '3fd66200f964a520f4e41ee3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(ven_id, client_id, client_secret, ver)

data4 = requests.get(url).json()
try:
    print(data4['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


# Exploring tips in Venues

In [36]:
data4['response']['venue']['tips']['count']

19

In [45]:
lim=20
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(ven_id, client_id, client_secret, ver, lim)
data = requests.get(url).json()
data

{'meta': {'code': 200, 'requestId': '5f0d80bc20a61b42e0170081'},
 'response': {'venue': {'id': '3fd66200f964a520f4e41ee3',
   'name': 'Ecco',
   'contact': {'phone': '2122277074',
    'formattedPhone': '(212) 227-7074',
    'facebook': '496751683729009',
    'facebookUsername': 'EccoRestaurant',
    'facebookName': 'Ecco'},
   'location': {'address': '124 Chambers St',
    'crossStreet': 'btwn Church St & W Broadway',
    'lat': 40.71533713859952,
    'lng': -74.00884766217825,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.71533713859952,
      'lng': -74.00884766217825},
     {'label': 'entrance', 'lat': 40.715202, 'lng': -74.008779}],
    'postalCode': '10007',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['124 Chambers St (btwn Church St & W Broadway)',
     'New York, NY 10007',
     'United States']},
   'canonicalUrl': 'https://foursquare.com/v/ecco/3fd66200f964a520f4e41ee3',
   'categories': [{'i

In [59]:
tips = data['response']['venue']['tips']['groups']
tip = tips[0]['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'entities', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [63]:
dftp = json_normalize(tip)
dftp = dftp[['text', 'agreeCount', 'disagreeCount', 'id']]
dftp

<ipython-input-63-495e2efdb2e5>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dftp = json_normalize(tip)


,text,agreeCount,disagreeCount,id
0,Old world comfort where you can enjoy veal cho...,2,0,4c7ac41f278eb713ae245c80


# Exploring Locations

In [64]:
lat = 40.715337
lng = -74.008848

In [65]:
url ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(client_id,client_secret,lat,lng,ver,radius,lim)
url

'https://api.foursquare.com/v2/venues/explore?client_id=ITKVISZUN50K5OIU5THJ0A330ABDVLFMOSX4NUB41ESFVYSA&client_secret=MACPFVUEFU3BEHD4YZREQUBDEBFX0JFOPQ0YLIEBRHWQ0ELC&ll=40.715337,-74.008848&v=20180604&radius=500&limit=20'

In [67]:
results = requests.get(url).json()
print('There are {} points of interests around Ecco restaurant.'.format(len(results['response']['groups'][0]['items'])))

There are 20 points of interests around Ecco restaurant.


In [68]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d5f24ec09484500079aee00',
  'name': 'Los Tacos No. 1',
  'location': {'address': '136 Church St',
   'lat': 40.714267,
   'lng': -74.008756,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.714267,
     'lng': -74.008756}],
   'distance': 119,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['136 Church St',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5d5f24ec09484500079aee00-0'}

In [72]:
df_xpl = json_normalize(items) # flatten JSON

# filter columns
fil_cols = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
df_xpl = df_xpl.loc[:, fil_cols]

# filter the category for each row
df_xpl['venue.categories'] = df_xpl.apply(get_category_type, axis=1)

# clean columns
df_xpl.columns = [col.split('.')[-1] for col in df_xpl.columns]

df_xpl.head()

<ipython-input-72-e7088866932e>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_xpl = json_normalize(items) # flatten JSON


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United Sta...",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839...",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007,...",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051...",202,10007,US,New York,NY,United States,[83 Murray St (btwn Greenwich St & W Broadway)...,btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
3,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",at Church St,NaN,4c154c9a77cea593c401d260
4,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.71577306392837...",88,10007,US,New York,NY,United States,[148 Chambers St (btwn West Broadway & Hudson ...,btwn West Broadway & Hudson St,NaN,4adcf23cf964a520cc6221e3


In [77]:
loc_m = folium.Map(location=[lat,lng], zoom_start=15)

folium.CircleMarker(
    [lat,lng],
    popup='Ecco',
    radius=12,
    fill=True,
    fill_color='red',
    fill_opacity=0.2).add_to(loc_m)

for lat,lng,label in zip(df_xpl.lat,df_xpl.lng,df_xpl.categories):
    folium.CircleMarker(
        [lat,lng],
        radius=8,
        popup=label,
        fill=True,
        fill_color='yellow',
        fill_opacity=0.4).add_to(loc_m)

loc_m